### installation 

In [2]:
! pip install datasets transformers rouge-score nltk

!git clone https://github.com/xmu-xiaoma666/External-Attention-pytorch.git ext_attns

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### About dataset 

In [4]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 11334
    })
})

In [6]:
print(f'train: {len(tokenized_datasets["train"])}')
print(f'val:   {len(tokenized_datasets["validation"])}')
print(f'test:  {len(tokenized_datasets["test"])}')

train: 204045
val:   11332
test:  11334


In [13]:
tr_max=20000
tokenized_datasets['train'][:tr_max

odict_keys(['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'])

In [210]:
tokenized_datasets['train']

Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
    num_rows: 204045
})

In [217]:
4000/204045

0.01960351883163028

### FT on small dataset 

|i|attention | used|
|---|--------| ----|
|0  |UFOAttention| ✔|
|1  | AFT_FULL   |x|
|2  | MUSEAttention|✔ |
|3  | EMSA|     x|
|4  |SimplifiedScaledDotProductAttention|✔ |
|5  |ScaledDotProductAttention|✔ |
|6  |ExternalAttention|✔ |

In [1]:
import os
ext_path = os.path.abspath('./ext_attns/')
import sys
sys.path.append(ext_path)
# list
from model.attention.UFOAttention import *
from model.attention.AFT import AFT_FULL
from model.attention.MUSEAttention import MUSEAttention
from model.attention.EMSA import EMSA
from model.attention.SimplifiedSelfAttention import SimplifiedScaledDotProductAttention
from model.attention.SelfAttention import ScaledDotProductAttention
from model.attention.ExternalAttention import ExternalAttention
d_model = 768 
n_head  = 12  
ufo = UFOAttention(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head)
aft_full = AFT_FULL(d_model=d_model, n=n_head)
ma = MUSEAttention(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head)
emsa = EMSA(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head,H=n_head,W=n_head,ratio=2,apply_transform=True)
ssa = SimplifiedScaledDotProductAttention(d_model=d_model, h=n_head)
sa = ScaledDotProductAttention(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head)
ea = ExternalAttention(d_model=d_model,S=8) #v

attn_li = [ufo,aft_full,ma,emsa,ssa,sa,ea]

import transformers

# print(transformers.__version__)

model_checkpoint = "t5-small"
model_checkpoint = 'facebook/bart-base'

from datasets import load_dataset, load_metric

raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")

# raw_datasets["train"][0]

import datasets

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# ?
# with tokenizer.as_target_tokenizer():
#     print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

batch_size = 3 #16

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-3d31b94c15be5d86.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-2e154596361058e4.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-f7a9e643055dbd7b.arrow


In [2]:
trvaltest = tokenized_datasets['train'].train_test_split(test_size=0.004,train_size=0.005) # 800,1080
# trvaltest = tokenized_datasets['train'].train_test_split(test_size=0.004,train_size=0.01) # 800,2080
# trvaltest = tokenized_datasets['train'].train_test_split(test_size=0.004,train_size=0.02) # 800,4080
tr = trvaltest['train']
valtest = trvaltest['test'].train_test_split(test_size=0.5,train_size=0.5) # 208, 208
# valtest = trvaltest['test'].train_test_split(test_size=0.5,train_size=0.5) # 408, 408
# valtest = trvaltest['test'].train_test_split(test_size=0.05,train_size=0.05)
val = valtest['train']
te = valtest['test']

del trvaltest
del valtest

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-914b7a1a143d71db.arrow and /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-37ec39fca056d44f.arrow


In [4]:
from torch import nn
batch_size = 16
evals = {}
for i,attn in enumerate(attn_li[:]):
    not_allowed = ('AFT_FULL','EMSA')
    print(attn)
    if attn.__str__().startswith(not_allowed):
        continue
    
    
    import os
    model_name = attn.__str__().split('(')[0]
    # model_name = model_checkpoint.split("/")[-1] + model_name
    saved_path = os.path.join('./output',model_name)
    
    args = Seq2SeqTrainingArguments(
        saved_path,
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=15,
        predict_with_generate=True,
        fp16=True,
        save_strategy='epoch',
        load_best_model_at_end=True,
    #     push_to_hub=True,
    )

    class tel2lin1(nn.Module):
        def __init__(self):
            super(tel2lin1,self).__init__()
            # self.tel = nn.TransformerEncoderLayer(d_model=768,nhead=3,batch_first=True)
            self.tel = attn
            self.lin = nn.Linear(in_features=768,out_features=50265, bias=False)
            
        def forward(self,x):
            if self.tel.__str__().startswith('ExternalAttention'):
                x = self.tel(x)
            else:
                x = self.tel(x,x,x)
            x = self.lin(x)
            return x
    model.lm_head = tel2lin1()

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tr,
        eval_dataset=val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    evals[f'{model_name}'] = trainer.evaluate(eval_dataset=te)

UFOAttention(
  (fc_q): Linear(in_features=768, out_features=768, bias=True)
  (fc_k): Linear(in_features=768, out_features=768, bias=True)
  (fc_v): Linear(in_features=768, out_features=768, bias=True)
  (fc_o): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 240
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,10.497597,0.000000,0.000000,0.000000,0.000000,4.000000
2,No log,9.946655,0.000000,0.000000,0.000000,0.000000,4.924000
3,No log,9.249656,0.000000,0.000000,0.000000,0.000000,5.000000
4,No log,8.560993,0.000000,0.000000,0.000000,0.000000,5.000000
5,No log,8.102303,0.035000,0.000000,0.035000,0.035000,4.000000
6,No log,7.837111,7.376000,0.000000,7.352500,7.225000,9.007400
7,No log,7.673697,0.035000,0.000000,0.035000,0.035000,4.000000
8,No log,7.506831,0.000000,0.000000,0.000000,0.000000,4.995100
9,No log,7.395956,0.000000,0.000000,0.000000,0.000000,4.980400
10,No log,7.315826,0.000000,0.000000,0.000000,0.000000,5.000000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 64
Saving model checkpoint to ./output/UFOAttention/checkpoint-16
Configuration saved in ./output/UFOAttention/checkpoint-16/config.json
Model weights saved in ./output/UFOAttention/checkpoint-16/pytorch_model.bin
tokenizer config file saved in ./output/UFOAttention/checkpoint-16/tokenizer_config.json
Special tokens file saved in ./output/UFOAttention/checkpoint-16/special_tokens_map.json
Deleting older checkpoint [output/UFOAttention/checkpoint-80] due to args.save_total_limit
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The foll

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


ExternalAttention(
  (mk): Linear(in_features=768, out_features=8, bias=False)
  (mv): Linear(in_features=8, out_features=768, bias=False)
  (softmax): Softmax(dim=1)
)


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 240


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,10.824655,7.376000,0.000000,7.352500,7.225000,8.000000
2,No log,10.824228,7.376000,0.000000,7.352500,7.225000,8.000000
3,No log,10.823728,7.376000,0.000000,7.352500,7.225000,8.000000
4,No log,10.823125,7.376000,0.000000,7.352500,7.225000,8.000000
5,No log,10.822416,7.376000,0.000000,7.352500,7.225000,8.000000
6,No log,10.821635,7.376000,0.000000,7.352500,7.225000,8.000000
7,No log,10.820818,7.376000,0.000000,7.352500,7.225000,8.000000
8,No log,10.819997,7.376000,0.000000,7.352500,7.225000,8.000000
9,No log,10.819201,7.376000,0.000000,7.352500,7.225000,8.000000
10,No log,10.818477,7.376000,0.000000,7.352500,7.225000,8.000000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 64
Saving model checkpoint to ./output/ExternalAttention/checkpoint-16
Configuration saved in ./output/ExternalAttention/checkpoint-16/config.json
Model weights saved in ./output/ExternalAttention/checkpoint-16/pytorch_model.bin
tokenizer config file saved in ./output/ExternalAttention/checkpoint-16/tokenizer_config.json
Special tokens file saved in ./output/ExternalAttention/checkpoint-16/special_tokens_map.json
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set  don't have a corresponding ar

In [26]:
import pickle
with open("attns.pickle","wb") as fw:
    pickle.dump(evals, fw)

In [27]:
with open('attns.pickle','rb') as fw:
    data = pickle.load(fw)

In [28]:
data

{'UFOAttention': {'eval_loss': 7.220878601074219,
  'eval_rouge1': 0.0,
  'eval_rouge2': 0.0,
  'eval_rougeL': 0.0,
  'eval_rougeLsum': 0.0,
  'eval_gen_len': 5.0,
  'eval_runtime': 12.9911,
  'eval_samples_per_second': 31.483,
  'eval_steps_per_second': 0.539,
  'epoch': 15.0},
 'MUSEAttention': {'eval_loss': 6.988565921783447,
  'eval_rouge1': 8.2008,
  'eval_rouge2': 0.0585,
  'eval_rougeL': 7.7756,
  'eval_rougeLsum': 7.7983,
  'eval_gen_len': 16.4719,
  'eval_runtime': 22.3288,
  'eval_samples_per_second': 18.317,
  'eval_steps_per_second': 0.313,
  'epoch': 15.0},
 'SimplifiedScaledDotProductAttention': {'eval_loss': 6.916492938995361,
  'eval_rouge1': 7.1489,
  'eval_rouge2': 0.1581,
  'eval_rougeL': 6.4239,
  'eval_rougeLsum': 6.5806,
  'eval_gen_len': 15.8093,
  'eval_runtime': 21.9655,
  'eval_samples_per_second': 18.62,
  'eval_steps_per_second': 0.319,
  'epoch': 15.0},
 'ScaledDotProductAttention': {'eval_loss': 7.626367092132568,
  'eval_rouge1': 0.0,
  'eval_rouge2': 0.0

In [25]:
evals

{'UFOAttention': {'eval_loss': 7.220878601074219,
  'eval_rouge1': 0.0,
  'eval_rouge2': 0.0,
  'eval_rougeL': 0.0,
  'eval_rougeLsum': 0.0,
  'eval_gen_len': 5.0,
  'eval_runtime': 12.9911,
  'eval_samples_per_second': 31.483,
  'eval_steps_per_second': 0.539,
  'epoch': 15.0},
 'MUSEAttention': {'eval_loss': 6.988565921783447,
  'eval_rouge1': 8.2008,
  'eval_rouge2': 0.0585,
  'eval_rougeL': 7.7756,
  'eval_rougeLsum': 7.7983,
  'eval_gen_len': 16.4719,
  'eval_runtime': 22.3288,
  'eval_samples_per_second': 18.317,
  'eval_steps_per_second': 0.313,
  'epoch': 15.0},
 'SimplifiedScaledDotProductAttention': {'eval_loss': 6.916492938995361,
  'eval_rouge1': 7.1489,
  'eval_rouge2': 0.1581,
  'eval_rougeL': 6.4239,
  'eval_rougeLsum': 6.5806,
  'eval_gen_len': 15.8093,
  'eval_runtime': 21.9655,
  'eval_samples_per_second': 18.62,
  'eval_steps_per_second': 0.319,
  'epoch': 15.0},
 'ScaledDotProductAttention': {'eval_loss': 7.626367092132568,
  'eval_rouge1': 0.0,
  'eval_rouge2': 0.0

In [7]:
from torch import nn
batch_size = 16
evalbase = {}
for i,attn in enumerate(['basic']):
    
    import os
    model_name = attn
    # model_name = model_checkpoint.split("/")[-1] + model_name
    saved_path = os.path.join('./output',model_name)
    
    args = Seq2SeqTrainingArguments(
        saved_path,
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=15,
        predict_with_generate=True,
        fp16=True,
        save_strategy='epoch',
        load_best_model_at_end=True,
    #     push_to_hub=True,
    )


    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tr,
        eval_dataset=val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    evals[f'{model_name}'] = trainer.evaluate(eval_dataset=te)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 240


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,10.815116,7.376000,0.000000,7.352500,7.225000,8.000000
2,No log,10.813450,7.376000,0.000000,7.352500,7.225000,8.000000
3,No log,10.811693,7.376000,0.000000,7.352500,7.225000,8.000000
4,No log,10.809827,7.376000,0.000000,7.352500,7.225000,8.000000
5,No log,10.807894,7.376000,0.000000,7.352500,7.225000,8.000000
6,No log,10.805979,7.376000,0.000000,7.352500,7.225000,8.000000
7,No log,10.804138,7.376000,0.000000,7.352500,7.225000,8.000000
8,No log,10.802417,7.376000,0.000000,7.352500,7.225000,8.000000
9,No log,10.800825,7.376000,0.000000,7.352500,7.225000,8.000000
10,No log,10.799348,7.376000,0.000000,7.352500,7.225000,8.000000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 64
Saving model checkpoint to ./output/basic/checkpoint-16
Configuration saved in ./output/basic/checkpoint-16/config.json
Model weights saved in ./output/basic/checkpoint-16/pytorch_model.bin
tokenizer config file saved in ./output/basic/checkpoint-16/tokenizer_config.json
Special tokens file saved in ./output/basic/checkpoint-16/special_tokens_map.json
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have be

### evals 

In [15]:
evals.update(evalbase)

In [19]:
import pandas as pd
pd.DataFrame(evals).T

,eval_loss,eval_rouge1,eval_rouge2,eval_rougeL,eval_rougeLsum,eval_gen_len,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
UFOAttention,7.220879,0.0000,0.0000,0.0000,0.0000,5.0000,12.9911,31.483,0.539,15.0
MUSEAttention,6.988566,8.2008,0.0585,7.7756,7.7983,16.4719,22.3288,18.317,0.313,15.0
SimplifiedScaledDotProductAttention,6.916493,7.1489,0.1581,6.4239,6.5806,15.8093,21.9655,18.620,0.319,15.0
ScaledDotProductAttention,7.626367,0.0000,0.0000,0.0000,0.0000,10.2812,19.9632,20.488,0.351,15.0
ExternalAttention,10.816548,8.0700,0.0000,8.0960,7.8017,8.0000,15.1313,27.030,0.463,15.0
basic,10.795363,8.0700,0.0000,8.0960,7.8017,8.0000,14.9132,27.425,0.469,15.0
